based on 

https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
from custom_tensorboard import TrainValTensorBoard

Using TensorFlow backend.


In [3]:
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

In [4]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [5]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [6]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [7]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix)
    #trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix)
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [8]:
train_x, train_y, test_x, test_y = load_dataset('../data/HAR/UCI_HAR_Dataset/')

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [9]:
train_x.shape

(7352, 128, 9)

In [10]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [11]:
epochs = 25
batch_size = 1000

keras imports

In [41]:
from keras import Model
from keras.layers import Lambda, Input, Dropout, Flatten, LSTM, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from time import time
from keras import optimizers

In [13]:
adam = optimizers.adam(lr=0.01)

In [23]:
name = 'LSTM'

model = Sequential(name =name)
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()
tensorboard = TensorBoard(log_dir="logs/{}".format(name + '_' + str(time())))
model.fit(train_x, 
          train_y, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.1,
          verbose=True,
         callbacks = [tensorboard],
)
#Input
x = Input((n_timesteps, n_features))

#LSTM
lstm_1 = LSTM(100, input_shape=(n_timesteps,n_features))(x)
lstm_2 = Dropout(0.5)(lstm_1)
lstm_3 = Dense(100, activation='relu')(lstm_2)
lstm_4 = Dense(n_outputs, activation='softmax', name='lstm_out')(lstm_3)

model = Model(x, lstm_4, name= 'LSTM' )
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir="logs/{}".format(name + '_' + str(time())))
model.fit(train_x, 
          train_y, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.1,
          verbose=True,
         callbacks = [tensorboard])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               44000     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_91 (Dense)             (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________
Train on 6616 samples, validate on 736 samples
Epoch 1/25
6616/6616 [==============================] - 4s 651us/step - loss: 1.7790 - acc: 0.2013 - val_loss: 1.7740 - val_acc: 0.2092
Epoch 2/25
6616/6616 [==============================] - 3s 521us/step - loss: 1.7733 - 

### Dense network approach

some comments about this architecture:

- Note that the unstack dim is the feature dimension

In [24]:
name = 'dense'

#Input
x = Input((n_timesteps, n_features))
#Dense
dense_1 = Lambda(lambda x: K.tf.unstack(x, axis=2))(x)
dense_2 = [Dense(20)(x) for x in dense_1]
dense_3 = Lambda(lambda x: K.stack(x, axis=2))(dense_2)
dense_4 = Dropout(0.1)(dense_3)
dense_5 = Flatten()(dense_4)
dense_6 = Dense(250, activation = 'relu')(dense_5)
dense_7 = Dense(20, activation = 'relu')(dense_6)
dense_8 = Dense(n_outputs, activation='softmax')(dense_7)


model = Model(x, dense_8, name ='dense')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
tensorboard = TensorBoard(log_dir="logs/{}".format(name + '_' + str(time())))
model.fit(train_x, 
          train_y, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.1,
          verbose=True,
         callbacks = [tensorboard])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 128, 9)       0                                            
__________________________________________________________________________________________________
lambda_15 (Lambda)              [(None, 128), (None, 0           input_10[0][0]                   
__________________________________________________________________________________________________
dense_93 (Dense)                (None, 20)           2580        lambda_15[0][0]                  
__________________________________________________________________________________________________
dense_94 (Dense)                (None, 20)           2580        lambda_15[0][1]                  
__________________________________________________________________________________________________
dense_95 (

# Hybrid ensemble

In [51]:
name = 'ens'

#Input
x = Input((n_timesteps, n_features))

#LSTM
lstm_1 = LSTM(100, input_shape=(n_timesteps,n_features))(x)
lstm_2 = Dropout(0.5)(lstm_1)
lstm_3 = Dense(100, activation='relu')(lstm_2)
lstm_4 = Dense(n_outputs, activation='softmax', name='lstm_out')(lstm_3)
#Dense
dense_1 = Lambda(lambda x: K.tf.unstack(x, axis=2))(x)
dense_2 = [Dense(20)(x) for x in dense_1]
dense_3 = Lambda(lambda x: K.stack(x, axis=2))(dense_2)
dense_4 = Dropout(0.1)(dense_3)
dense_5 = Flatten()(dense_4)
dense_6 = Dense(250, activation = 'relu')(dense_5)
dense_7 = Dense(20, activation = 'relu')(dense_6)
dense_8 = Dense(n_outputs, activation='softmax')(dense_7)

ens_1 = Concatenate(axis=1)([lstm_3,dense_7])
ens_2 = Dense(n_outputs, activation='softmax')(ens_1)

In [52]:
model = Model(x, ens_2)

In [53]:
model.compile(optimizer='adam', 
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir="logs/{}".format(name + '_' + str(time())))

model.fit(train_x, 
          train_y, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.1,
          verbose=True,
         callbacks = [tensorboard])

Train on 6616 samples, validate on 736 samples
Epoch 1/25
6616/6616 [==============================] - 5s 758us/step - loss: 1.5578 - acc: 0.4022 - val_loss: 1.2925 - val_acc: 0.5557
Epoch 2/25
6616/6616 [==============================] - 3s 524us/step - loss: 1.2221 - acc: 0.5289 - val_loss: 1.0492 - val_acc: 0.5815
Epoch 3/25
6616/6616 [==============================] - 3s 526us/step - loss: 1.0223 - acc: 0.5859 - val_loss: 0.8518 - val_acc: 0.6807
Epoch 4/25
6616/6616 [==============================] - 3s 524us/step - loss: 0.9265 - acc: 0.6375 - val_loss: 0.7999 - val_acc: 0.6440
Epoch 5/25
6616/6616 [==============================] - 3s 524us/step - loss: 0.8255 - acc: 0.6911 - val_loss: 0.7274 - val_acc: 0.7120
Epoch 6/25
6616/6616 [==============================] - 3s 527us/step - loss: 0.7065 - acc: 0.7406 - val_loss: 0.6016 - val_acc: 0.7405
Epoch 7/25
4000/6616 [=================>............] - ETA: 1s - loss: 0.6236 - acc: 0.7823